In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
import numpy as np
import os

# --- CONFIG ---
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 10

# --- Dataset Path ---
data_dir = r'C:\Hope\AI Course Tamil\8.Deep Learning\New folder (3)\Dataset'

# --- Image Preprocessing ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# --- Save class names for prediction use ---
class_names = list(train_generator.class_indices.keys())

# --- Build the CNN Model ---
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# --- Compile Model ---
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Train Model ---
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# --- Save Model ---
model.save("nemo_fish_name_model.h5")

Found 2666 images belonging to 9 classes.
Found 661 images belonging to 9 classes.


C:\Anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.5302 - loss: 1.5156 - val_accuracy: 0.9259 - val_loss: 0.3449
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.9483 - loss: 0.2963 - val_accuracy: 0.9349 - val_loss: 0.3402
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step - accuracy: 0.9515 - loss: 0.2400 - val_accuracy: 0.8850 - val_loss: 0.4207
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.9670 - loss: 0.1595 - val_accuracy: 0.9168 - val_loss: 0.3614
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.9718 - loss: 0.1398 - val_accuracy: 0.8971 - val_loss: 0.5855
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.9742 - loss: 0.1421 - val_accuracy: 0.8185 - val_loss: 0.6458
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.9698 - loss: 0.1292 - val_accuracy: 0.9531 - val_loss: 0.2829
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.9806 - loss: 0.0883 - val_accuracy: 0.8744 - v

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# --- CONFIG ---
IMG_SIZE = (150, 150)
model_path = "nemo_fish_name_model.h5"
image_folder = r"C:\Hope\AI Course Tamil\8.Deep Learning\test_images"

# --- Manually specify class names (must match training folders) ---
class_names = ["bloat", "bubbles", "deb_flo", "dory", "gill",
               "gurgle", "jacques", "marlin", "nemo", "peach"]

supported_exts = [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"]

# --- Load model ---
model = tf.keras.models.load_model(model_path)

# --- Predict all images in folder ---
image_files = [f for f in os.listdir(image_folder) if os.path.splitext(f)[1].lower() in supported_exts]

if not image_files:
    print("❌ No image files found in the folder.")
else:
    for filename in image_files:
        try:
            full_path = os.path.join(image_folder, filename)
            img = load_img(full_path, target_size=IMG_SIZE)
            img_array = img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            pred = model.predict(img_array)
            predicted_index = np.argmax(pred)
            predicted_class = class_names[predicted_index]
            confidence = pred[0][predicted_index] * 100

            print(f"🖼 {filename} ➜ 🐟 {predicted_class} ({confidence:.2f}%)")

        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
🖼 GBU.jpeg ➜ 🐟 dory (96.27%)
